Scripts

In [6]:
from newsapi import NewsApiClient
import datetime
import time
import pandas as pd
import os
from newspaper import Article
import sys

In [ ]:
# Initialize NewsAPI Client
api_key = ""  # Replace with API key
newsapi = NewsApiClient(api_key=api_key)

# Input and output file paths
input_file = "news_articles_election_candidates_expanded.csv"
full_content_file = "news_articles_election_candidates_full_content.csv"

# Function to fetch full article content using newspaper3k
def fetch_full_content(article_url):
    try:
        article = Article(article_url)
        article.download()
        article.parse()
        return article.text  # Return the full article text
    except Exception as e:
        return None  # Return None if there is an error

# Set new date range
start_date = datetime.date(2024, 10, 12)  # Continue from where the previous script left off
end_date = datetime.date(2024, 8, 1)  # Adjust end date as needed for backward collection

# Prepare to store results
articles_data = []

# Expanded search queries
queries = [
    "2024 Presidential election",
    "US election AND (Donald Trump OR Kamala Harris)",
    "Biden administration AND 2024 election",
    "(Donald Trump OR Trump) AND 2024 election",
    "(Kamala Harris OR Harris) AND 2024 election",
    "Campaign financing AND 2024 election",
    "Voter turnout AND 2024 election",
    "Presidential debate AND 2024 election",
    "(Donald Trump OR Trump) AND rally AND 2024",
    "(Kamala Harris OR Harris) AND speech AND 2024",
    "(Donald Trump OR Kamala Harris) AND 2024 election",
    "(Trump OR Harris) AND campaign AND 2024 election"
]

# Track API request count to avoid exceeding limits
request_count = 0
max_requests = 100  # Free-tier daily API limit

# Load existing expanded and full content CSV files
existing_expanded_data = pd.read_csv(input_file) if os.path.exists(input_file) else pd.DataFrame()
existing_full_content_data = pd.read_csv(full_content_file) if os.path.exists(full_content_file) else pd.DataFrame()

# Track URLs to avoid duplicates
existing_urls = set(existing_expanded_data["url"]) if not existing_expanded_data.empty else set()
processed_urls = set(existing_full_content_data["url"]) if not existing_full_content_data.empty else set()

# Fetch new articles from NewsAPI
current_date = start_date
while current_date >= end_date:
    # Convert date to string for API
    date_str = current_date.strftime('%Y-%m-%d')
    print(f"Fetching articles for {date_str}...")

    for query in queries:
        try:
            # Check if API limit is reached
            if request_count >= max_requests:
                print("Reached API limit for the day. Exiting script.")
                sys.exit()

            # Fetch articles for the current query and date
            response = newsapi.get_everything(
                q=query,
                from_param=date_str,
                to=date_str,
                language="en",
                sort_by="relevancy",  # Fetch relevant articles
                page_size=100  # Max articles per API call
            )

            # Increment request count
            request_count += 1

            if response.get('status') != 'ok':
                print(f"API error: {response.get('message')}")
                sys.exit()

            # Process the articles
            if response.get('articles'):
                for article in response['articles']:
                    # Only add new articles that are not already saved
                    if article['url'] not in existing_urls:
                        articles_data.append({
                            "query": query,  # Include the query used for tracking
                            "source": article['source']['name'],
                            "author": article['author'],
                            "title": article['title'],
                            "description": article['description'],
                            "url": article['url'],
                            "published_at": article['publishedAt'],
                            "content": article['content']
                        })
                        # Add the URL to the set of existing URLs
                        existing_urls.add(article['url'])
        except Exception as e:
            # Log the error to a file
            with open("error_log.txt", "a") as log_file:
                log_file.write(f"Error fetching articles for {query} on {date_str}: {e}\n")
            print(f"Error fetching articles for {query} on {date_str}: {e}")
    
    # Move to the previous day
    current_date -= datetime.timedelta(days=1)

    # Avoid hitting API limits by adding a small delay between requests
    time.sleep(1)

# Save new articles to the expanded CSV file
if articles_data:
    new_data_df = pd.DataFrame(articles_data)
    new_data_df.to_csv(input_file, mode='a', header=not os.path.exists(input_file), index=False)
    print(f"Appended {len(new_data_df)} new articles to '{input_file}'.")

# Extract full content for new articles only
new_urls = {article["url"] for article in articles_data}  # URLs of newly fetched articles
urls_to_process = new_urls - processed_urls  # Exclude already processed URLs

if urls_to_process:
    full_content_data = []
    for url in urls_to_process:
        print(f"Fetching full content for {url}...")
        full_content = fetch_full_content(url)
        if full_content:
            full_content_data.append({
                "url": url,
                "full_content": full_content
            })
        else:
            print(f"Failed to fetch full content for {url}")

    # Append full content to the full content CSV file
    if full_content_data:
        full_content_df = pd.DataFrame(full_content_data)
        full_content_df.to_csv(full_content_file, mode='a', header=not os.path.exists(full_content_file), index=False)
        print(f"Appended {len(full_content_df)} new full content articles to '{full_content_file}'.")
else:
    print("No new articles to process for full content.")



In [ ]:
# Load the CSV file
file_path = 'news_articles_election_candidates_expanded.csv'
data = pd.read_csv(file_path)

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9610 entries, 0 to 9609
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query         9610 non-null   object
 1   source        9610 non-null   object
 2   author        9093 non-null   object
 3   title         9607 non-null   object
 4   description   9598 non-null   object
 5   url           9610 non-null   object
 6   published_at  9610 non-null   object
 7   content       9610 non-null   object
dtypes: object(8)
memory usage: 600.8+ KB


In [9]:
file_path = 'news_articles_election_candidates_full_content.csv'  # Replace with your file path
data_full_content = pd.read_csv(file_path)

data_full_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9610 entries, 0 to 9609
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query         9610 non-null   object
 1   source        9610 non-null   object
 2   author        9093 non-null   object
 3   title         9607 non-null   object
 4   description   9598 non-null   object
 5   url           9610 non-null   object
 6   published_at  9610 non-null   object
 7   content       9610 non-null   object
 8   full_content  7556 non-null   object
dtypes: object(9)
memory usage: 675.8+ KB


In [10]:
file_size = os.path.getsize('news_articles_election_candidates_full_content.csv') / (1024 * 1024)
print(f"File size: {file_size:.2f} MB")


File size: 47.67 MB


Remove entries with missing full content

In [12]:
# Remove rows with missing full content
data_full_content = data_full_content.dropna(subset=['full_content'])

# Check the updated DataFrame info
data_full_content.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7556 entries, 0 to 9609
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query         7556 non-null   object
 1   source        7556 non-null   object
 2   author        7084 non-null   object
 3   title         7553 non-null   object
 4   description   7548 non-null   object
 5   url           7556 non-null   object
 6   published_at  7556 non-null   object
 7   content       7556 non-null   object
 8   full_content  7556 non-null   object
dtypes: object(9)
memory usage: 590.3+ KB


In [13]:
# Save the cleaned dataframe to a new CSV file
data_full_content.to_csv('news_articles_election_candidates_full_content_cleaned.csv', index=False)


In [14]:
print("Number of rows (articles):", data_full_content.shape[0])
print("Number of variables (columns):", data_full_content.shape[1])


Number of rows (articles): 7556
Number of variables (columns): 9


In [15]:
# Calculate the number of unique media outlets
unique_media_outlets = data_full_content['source'].nunique()

# Display the number of unique media outlets
print(f"Number of unique media outlets: {unique_media_outlets}")


Number of unique media outlets: 454
